In [ ]:
!pip list


In [ ]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)



In [ ]:
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
ds = tfds.load('fashion_mnist', split="train")

In [ ]:
np.squeeze(ds.as_numpy_iterator().next()["image"]).shape

In [ ]:
dataiterator = ds.as_numpy_iterator()

In [ ]:
dataiterator.next()

In [ ]:
fig , ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4):
    batch = dataiterator.next()
    ax[idx].imshow(np.squeeze(batch["image"]))
    ax[idx].title.set_text(batch["label"])

In [ ]:
#scale image
def scale_images(data):
    image = data["image"]
    return image/255

map
cache
shuffle
batch
prefetch

In [ ]:
ds= tfds.load("fashion_mnist",split= "train")
# running the dataset through the scale_image preprocessing step
ds = ds.map(scale_images)
# cache the dataset for the batch
ds = ds.cache()
# shuffle it up
ds = ds.shuffle(60000)
# batch into 128 images per sample
ds= ds.batch(128)
# reduces the likelihood of bottlenecking 
ds = ds.prefetch(64)

In [ ]:
ds.as_numpy_iterator().next().shape

# Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , Dense , Flatten ,Reshape , LeakyReLU, Dropout , UpSampling2D

In [ ]:
def build_generator():
    model= Sequential()
    
    #block1 takes in random values and reshape it into 7x7x128
    model.add(Dense(7*7*128 , input_dim = 128))
    model.add(LeakyReLU(0.2))
    model.add(Reshape((7,7,128)))
    
    #upsampling block 1
    model.add(UpSampling2D())
    model.add(Conv2D(128,5,padding="same"))
    model.add(LeakyReLU(0.2))
    
    #upsampling block 2
    model.add(UpSampling2D())
    model.add(Conv2D(128,5,padding="same"))
    model.add(LeakyReLU(0.2))
    
    # down sampling block 1
    model.add(Conv2D(128,4,padding="same"))
    model.add(LeakyReLU(0.2))
    
    # down sampling block 2
    model.add(Conv2D(128,4,padding="same"))
    model.add(LeakyReLU(0.2))
    
    #Conv Layer to get to one Channel
    model.add(Conv2D(1,4,padding="same", activation='sigmoid'))
    
    
    
    return model

In [ ]:
generator = build_generator()
generator.summary()

In [ ]:
img=generator.predict(np.random.randn(4,128,1))
img.shape


In [ ]:
fig , ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in   enumerate(img):
    ax[idx].imshow(np.squeeze(img))
    ax[idx].title.set_text(idx)

In [ ]:
# BUILD DISCRIMINATOR

In [ ]:
def build_discriminator():
    model = Sequential()

    #first conv Block
    model.add(Conv2D(32,5,input_shape = (28,28,1)))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))

    #second conv Block
    model.add(Conv2D(64,5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))

    # third conv Block
    model.add(Conv2D(128,5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))

    # fourth conv Block
    model.add(Conv2D(256,5))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))

    #flatten then pass to dense Layer

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation="sigmoid"))



    return model

In [ ]:
discriminator= build_discriminator()
discriminator.summary()

In [ ]:
discriminator.predict(np.expand_dims(img,0))

# Construct Training Loop

setup Losses and Optimizers

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [ ]:
g_opt= Adam(learning_rate=0.0001)
d_opt = Adam(learning_rate=0.00001)
g_loss= BinaryCrossentropy()
d_loss = BinaryCrossentropy()

Build Subclassed Model

In [ ]:
# Import the base model class to subclass our training steps
from tensorflow.keras.models import Model 

In [ ]:
class FashionGAN(Model):
    def __init__(self, generator,discriminator , *args,**kwargs):
        # pass through args and kwards to base class
        super().__init__(*args, **kwargs)

        #create attributes for gen and disc
        self.generator = generator
        self.discriminator = discriminator


    def compile(self, g_opt,d_opt,g_loss,d_loss,*args, **kwargs):

        super().compile(*args, **kwargs)
        self.g_opt= g_opt
        self.d_opt= d_opt
        self.g_loss = g_loss
        self.d_loss = d_loss

    def train_step(self, batch):

        #get the real data
        real_images = batch
        fake_images = self.generator(tf.random.normal((128,128,1)), training=False)

        #train the discriminator
        with tf.GradientTape() as d_tape:

            #pass the real and fake images to the discriminator model
            yhat_real = self.discriminator(real_images, training= True)
            yhat_fake= self.discriminator(fake_images, training=True)
            yhat_realfake = tf.concat([yhat_real,yhat_fake],axis=0)

            #create Labels for real and fake images
            y_realfake = tf.concat([tf.zeros_like(yhat_real),tf.ones_like(yhat_fake)], axis=0)

            # Add some noise to the true outputs
            noise_real= 0.15*tf.random.uniform(tf.shape(yhat_real))
            noise_fake= -0.15*tf.random.uniform(tf.shape(yhat_fake))
            y_realfake += tf.concat([noise_real,noise_fake],axis=0)


            # Calculate loss

            total_d_loss= self.d_loss(y_realfake,yhat_realfake)

        # Apply backpropagation  -nn learn
        dgrad = d_tape.gradient(total_d_loss, self.discriminator.trainable_variables)
        self.d_opt.apply_gradients(zip(dgrad, self.discriminator.trainable_variables))

        #train the generator
        with tf.GradientTape() as g_tape:
            #generate some new images
            gen_images = self.generator(tf.random.normal((128,128,1)), training= True)
            #create the predicted labels
            predected_labels = self.discriminator(gen_images,training= False)
            #calculate loss -trick to training to fake out the discriminator
            total_g_loss= self.g_loss(tf.zeros_like(predected_labels), predected_labels)
        #apply backprop
        ggrad = g_tape.gradient(total_g_loss, self.generator.trainable_variables)
        self.g_opt.apply_gradients(zip(ggrad, self.generator.trainable_variables))

        return {"d_loss": total_d_loss , "g_loss":total_g_loss}



        

In [ ]:
#create instance of subclassed model

fashgan = FashionGAN(generator , discriminator)

In [ ]:
#compile the model 
fashgan.compile(g_opt,d_opt,g_loss,d_loss)

Build Callback


In [ ]:
import os 
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.callbacks import Callback

In [ ]:
class ModelMonitor(Callback):
    def __init__(self, num_img=3 , latent_dim =128):
        self.num_img = num_img
        self.latent_dim = latent_dim
    
    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.uniform((self.num_img,self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img= array_to_img(generated_images[i])
            img.save(os.path.join('images', f"generated_img_{epoch}_{i}.png"))
          

In [ ]:
monitor = ModelMonitor()

# Train


In [ ]:
import sys

import keras
import pandas as pd
import sklearn as sk
import scipy as sp
import tensorflow as tf
import platform

print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")

print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print(f"SciPy {sp.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")


In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Run on GPU if available
with tf.device('/GPU:0'):
    # Your model training code here
    hist = fashgan.fit(ds, epochs=1, callbacks=[ModelMonitor()])

In [ ]:
plt.suptitle('Loss')
plt.plot(hist.history["d_loss"],label="d_loss")
plt.plot(hist.history["g_loss"],label="g_loss")
plt.legend()
plt.show()

In [ ]:
imgs = generator.predict(tf.random.normal((16,128,1)))

In [ ]:
fig,ax = plt.subplots(ncols=4,nrows=4, figsize=(10,10))
for r in range(4):
    for c in range(4):
        ax[r][c].imshow(imgs[(r+1)*(c+1)-1])